In [1]:
import sys
sys.path.append("..")   # this allows us to import from sibling directory

from code_to_import.dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from code_to_import.Classifier_module import Classifier
from torch.autograd import Variable


In [2]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################
path_to_model = "../models/two_layers/twoLayerModel_weights_gpu_mumu0"

event_type = "mumu"  # could be ee, emu, mumu
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

In [39]:
# model = 0
model = Classifier()
model.load_state_dict(torch.load(path_to_model))
model.eval()

Classifier(
  (model): Sequential(
    (0): Linear(in_features=80, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=256, out_features=3, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
  )
)

In [40]:



data_object = ProductionModeDataset(file)
x = data_object.get_eval_data()


['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 'lbar_eta', 'l_phi', 'lbar_phi', '

In [41]:
weight = x[:,81]
target = x[:,80]
target = Variable(torch.from_numpy(target).type(torch.LongTensor))
y = np.transpose(x)
y = np.delete(y, [80, 81, 82], 0)
y = np.transpose(y)
input = Variable(torch.from_numpy(y).type(torch.Tensor))

In [43]:
model(input)[0,:]

tensor([1.4296, 2.1556, 2.6216], grad_fn=<SliceBackward>)

In [44]:
target[0]

tensor(2)

In [45]:
from skorch import NeuralNetClassifier
from sklearn.inspection import permutation_importance

In [46]:
nn = NeuralNetClassifier(model, max_epochs=0, lr=0.00, batch_size=5000)
nn.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Classifier(
    (model): Sequential(
      (0): Linear(in_features=80, out_features=256, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Linear(in_features=256, out_features=3, bias=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  ),
)

In [47]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [48]:
nn.score(input, target, sample_weight = weight)

0.7171375747173215

In [49]:
r = permutation_importance(nn, input, target, \
                           n_repeats=30, \
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{input_variables[i]:<8}         "
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")


ttbar_pt         0.094 +/- 0.003
wplus_phi         0.087 +/- 0.004
b_phi            0.086 +/- 0.002
llbar_delta_phi         0.084 +/- 0.004
wminus_phi         0.080 +/- 0.003
nunubar_delta_phi         0.079 +/- 0.004
tbar_phi         0.075 +/- 0.003
bnu_delta_phi         0.075 +/- 0.003
wminusbbar_delta_phi         0.075 +/- 0.003
lbarb_delta_phi         0.075 +/- 0.003
wplusb_delta_phi         0.074 +/- 0.003
l_phi            0.073 +/- 0.004
bbarnubar_delta_phi         0.073 +/- 0.003
bbbar_phi         0.073 +/- 0.004
bbar_phi         0.072 +/- 0.003
top_phi          0.071 +/- 0.003
nubar_phi         0.070 +/- 0.004
ttbar_phi         0.070 +/- 0.003
nunubar_phi         0.069 +/- 0.004
nu_phi           0.068 +/- 0.003
lbar_phi         0.067 +/- 0.003
b_eta            0.067 +/- 0.003
l_eta            0.067 +/- 0.003
lbbar_delta_phi         0.067 +/- 0.003
lnubar_delta_phi         0.067 +/- 0.003
lbarnu_delta_phi         0.065 +/- 0.003
bbbar_delta_phi         0.065 +/- 0.003
lbar_eta   